In [ ]:
import k3d
import numpy as np
from scipy import stats
import scipy.optimize as optimize

import sys
sys.path.insert(0, "..")

from Geometry.HALL import HALL
from Geometry.SM import SM

In [ ]:
# Analyze the July 10 2024 survey of the unloaded structure, using temporary bar ends (bottom and top supermodules)

# Define the survey area as a Hall that contains the supermodules.

class Room(HALL):
    # define a room containing the two supermodules (Bldg 157)

    # select the supermodule types used in the survey by setting the index
    super_modules = ['bottom','top']

    room_sms = []
    # place each supermodule in the room - its position and orientation placed roughly as indicated by the survey report
    # The survey team used the following cartesian coordinate system (rotated by about 90 degrees around the z axis):
    #  * same z axis as the WCTE SM coordinate systems
    #  * x axis is approximately parallel to the negative y axis of the WCTE SM coordinate systems
    #  * y axis is approximately parallel to the positive x axis of the WCTE SM coordinate systems
    # * the origin is at the centre of the overall support structure: 
    #  * the survey has the average z coordinate of targets at +/- 1533.9
    #  * this needs to be reduced by 75 mm to refer to the origin of the endcap super modules
    
    
    room_sms.append({'name': super_modules[0], 'kind': super_modules[0],
                    'loc': [0., 0., -1533.9 + 75],
                    'loc_sig': [10.0, 10.0, 10.0],
                    'rot_axes': 'ZYX',
                    'rot_angles': [np.pi/2., 0., 0.],
                    'rot_angles_sig': [0.2, 0.2, 0.2]})
    
    room_sms.append({'name': super_modules[1], 'kind': super_modules[1],
                    'loc': [0., 0., 1533.9 + 75],
                    'loc_sig': [10.0, 10.0, 10.0],
                    'rot_axes': 'ZYX',
                    'rot_angles': [np.pi/2., 0., 0.],
                    'rot_angles_sig': [0.2, 0.2, 0.2]})

    HALL.sms_design['room'] = room_sms


In [ ]:
# create the supermodules and place them in the room:

my_room = Room('my_room', kind='room', device_type=SM)

In [ ]:
# Survey data:

bottom_survey_data = [
    [-0.1624, 1.8477, -1.5343],
    [-0.7565, 1.6171, -1.5380],
    [-1.3720, 1.1060, -1.5365],
    [-1.8076, 0.4134, -1.5342],
    [-1.8471, -0.1635, -1.5335],
    [-1.5121, -0.9076, -1.5351],
    [-0.9715, -1.5021, -1.5381],
    [-0.4117, -1.8107, -1.5359],
    [0.1650, -1.8511, -1.5382],
    [0.7630, -1.6198, -1.5377],
    [1.3775, -1.1036, -1.5380],
    [1.8066, -0.4132, -1.5347],
    [1.8482, 0.1634, -1.5331],
    [1.5151, 0.9098, -1.5366],
    [0.9734, 1.5000, -1.5364],
    [0.4152, 1.8077, -1.5347]
]

top_survey_data = [
    [-0.1552, 1.8479, 1.5320],
    [-0.7513, 1.6222, 1.5309],
    [-1.3690, 1.1121, 1.5306],
    [-1.8045, 0.4225, 1.5335],
    [-1.8470, -0.1552, 1.5349],
    [-1.5167, -0.8991, 1.5324],
    [-0.9799, -1.4966, 1.5313],
    [-0.4224, -1.8080, 1.5301],
    [0.1562, -1.8515, 1.5286],
    [0.7548, -1.6242, 1.5312],
    [1.3698, -1.1107, 1.5289],
    [1.8045, -0.4217, 1.5336],
    [1.8470, 0.1544, 1.5334],
    [1.5171, 0.9020, 1.5311],
    [0.9828, 1.4930, 1.5344],
    [0.4220, 1.8044, 1.5320]
    ]

In [ ]:
# get one supermodule in the room 

ism = 0
sm = my_room.sms[ism]
survey_data = [bottom_survey_data, top_survey_data][ism]

# look at RMS of deviations of the target positions from survey from the design positions

sum2 = 0.
for i_target, target in enumerate(sm.targets):
    p = target.get_placement('design')
    location, direction_z = p['location'], p['direction_z']

    survey_location = np.array(survey_data[i_target])*1000. # convert to mm
    diff = np.subtract(location,survey_location)
    sum2 += np.dot(diff,diff)
rms = np.sqrt(sum2/len(sm.targets))
print('RMS of deviations of the true target positions compared to survey positions: %.2f mm' % rms)


In [ ]:
# reveal how the sm is placed in the room
print('true:', sm.get_placement('true'))

# initialize the estimated placements with the design values
sm.place_est = sm.place_design.copy()
print('est:', sm.get_placement('est'))

for target in sm.targets:
    target.place_est = target.place_design.copy()
    
for mpmt in sm.mpmts:
    mpmt.place_est = mpmt.place_design.copy()

In [ ]:
# Calculate sum of squares of residuals for all targets in sm 

def calc_dev2_all():
    sum2 = 0.
    for i_target, target in enumerate(sm.targets):
        p = target.get_placement('est')
        location, direction_z = p['location'], p['direction_z']

        survey_location = np.array(survey_data[i_target])*1000. # convert to mm
        diff = np.subtract(location,survey_location)
        sum2 += np.dot(diff,diff)
    return sum2

# parameters specify the sm placement
def calc_dev2_sm(params):
    loc = [params[i] for i in range(3)]
    rot_angles = [params[i] for i in range(3,6)]
    sm.place_est['loc'] = loc
    sm.place_est['rot_angles'] = rot_angles
    
    return calc_dev2_all()


In [ ]:
sum2 = calc_dev2_all()
print('all:',np.sqrt(sum2/len(sm.targets)))
print('est:', sm.get_placement('est'))

In [ ]:
# find optimal sm placement parameters - start from the current values stored in place_est
start = sm.place_est['loc'] + sm.place_est['rot_angles']
result = optimize.minimize(calc_dev2_sm, start)
print(result.message)
    
print('true:', sm.get_placement('true'))
print('est:', sm.get_placement('est'))

sum2 = calc_dev2_all()
print('rms of residuals:', np.sqrt(sum2/len(sm.targets)))

In [ ]:
# Show in 3D the deviations between est and survey (or between est and true) to illustrate how survey optimization proceeds 

# to keep mpmt numbering consistent with the full WCTE detector, we need to add an offset
first_target = {'bottom':0, 'top':16}[sm.name]
first_mpmt = {'bottom':0, 'barrel':21, 'top':85}[sm.name]

plot = k3d.plot()


origins = []
survey_vecs = []
vec_length = 100. # length of x,z vectors to show for each mpmt

survey_scale = 1. # scale up the length of deviations
color_survey = 0xff0000
color_mpmt = 0xabb2b9
color_target = 0xad4113

color_missing_mpmt = 0xa83273
missing_mpmts = [85, 91, 99]

n_point_mpmt = 8
indices_mpmt = []
for i in range(n_point_mpmt):
    indices_mpmt.append([i,(i+1)%n_point_mpmt])

n_fd_point_mpmt = 40
fd_indices_mpmt = []
for i in range(n_fd_point_mpmt):
    fd_indices_mpmt.append([i,(i+1)%n_fd_point_mpmt])

n_point_ft = 20 # for feedthroughs and C holes
indices_ft = []
for i in range(n_point_ft):
    indices_ft.append([i,(i+1)%n_point_ft])

# draw the extent of the mpmt baseplates
for i_mpmt,mpmt in enumerate(sm.mpmts):
    p = mpmt.get_placement('est')
    location = p['location']

    baseplate_points = np.array(mpmt.get_xy_points('est'),dtype=np.float32)
    color = color_mpmt
    if i_mpmt+first_mpmt in missing_mpmts:
        color = color_missing_mpmt
        
    indices = indices_mpmt
    if mpmt.kind == 'FD':
        indices = fd_indices_mpmt
    plt_baseplate = k3d.lines(baseplate_points, indices, indices_type='segment', color=color)
    plot += plt_baseplate

    if i_mpmt+first_mpmt not in missing_mpmts:
        if mpmt.kind != 'FD':
            feedthrough_points = np.array(mpmt.get_xy_points('est', feature='feedthrough'),dtype=np.float32)
            plt_feedthrough = k3d.lines(feedthrough_points, indices_ft, indices_type='segment', color=color_mpmt)
            plot += plt_feedthrough
        else:
            feedthrough1_points = np.array(mpmt.get_xy_points('est', feature='fd_feedthrough1'),dtype=np.float32)
            plt_feedthrough1 = k3d.lines(feedthrough1_points, indices_ft, indices_type='segment', color=color_mpmt)
            plot += plt_feedthrough1
            feedthrough2_points = np.array(mpmt.get_xy_points('est', feature='fd_feedthrough2'),dtype=np.float32)
            plt_feedthrough2 = k3d.lines(feedthrough2_points, indices_ft, indices_type='segment', color=color_mpmt)
            plot += plt_feedthrough2

    # k3d complains about the following not being float32!
    plt_text = k3d.text(str(i_mpmt+first_mpmt), position=location, reference_point='cc', size=1., label_box=False, color=color)
    plot += plt_text

# draw the targets
target_origins = []
if 1==1 and sm.targets is not None:
    for i_target, target in enumerate(sm.targets):
        
        p = target.get_placement('est')
        location, direction_x, direction_z = p['location'], p['direction_x'], p['direction_z']
        target_origins.append(location)
        text_location = [location[j] - 10.*direction_z[j] for j in range(3)]
        plt_text = k3d.text(str(i_target+first_target), position=text_location, reference_point='cc', size=1., label_box=False, color=color_target)
        plot += plt_text
    
#plot target locations and axis
target_locations = np.array(target_origins, dtype=np.float32)
plt_targets = k3d.points(positions=target_locations,
                        point_size=8.,
                        shader='3d',
                        color=color_target)
plot += plt_targets

# show the surveyed locations of the targets

target_surveys = []
for survey_datum in survey_data:
    target_surveys.append(np.array(survey_datum, dtype=np.float32)*1000.) # convert to mm

plt_fs = k3d.points(positions=target_surveys,
                    point_size=8.,
                    shader='3d',
                    color=color_survey)
plot += plt_fs

# show a vector from the design to the surveyed
survey_scale = 100. # scale up the length of deviations
for i in range(len(survey_data)):
    origins.append(target_locations[i])
    diff = np.subtract(target_surveys[i],target_locations[i])
    scaled_diff = list(diff*survey_scale)
    survey_vecs.append(scaled_diff)

plt_survey_vecs = k3d.vectors(origins=origins, vectors=survey_vecs, color=color_survey, head_size=250.)
plot += plt_survey_vecs

plot.display()

In [ ]:
top_placement = sm.get_placement('est')

In [ ]:
bottom_placement = sm.get_placement('est')

In [ ]:
print(np.arccos(np.dot(top_placement['direction_z'],bottom_placement['direction_z'])))
print(np.arccos(np.dot(top_placement['direction_x'],bottom_placement['direction_x'])))

In [ ]:
# save the top level device (and thereby the entire geometry)
my_room.save_file('my_room.geo')

In [ ]:
# read the saved device back again
from Geometry.Device import Device
new_room = Device.open_file('my_room.geo')